# Create Training Data for Reranking

In [ ]:
'''
Predict tracks for a playlist given its title only
Predict tracks for a playlist given its title and the first track
Predict tracks for a playlist given its title and the first 5 tracks
Predict tracks for a playlist given its first 5 tracks (no title)
Predict tracks for a playlist given its title and the first 10 tracks
Predict tracks for a playlist given its first 10 tracks (no title)
Predict tracks for a playlist given its title and the first 25 tracks
Predict tracks for a playlist given its title and 25 random tracks
Predict tracks for a playlist given its title and the first 100 tracks
Predict tracks for a playlist given its title and 100 random tracks
'''

In [ ]:
'''
variants=[1,5,10,25,100]
random=[25,100]
only title -> 1
1 track -> 1
5 tracks -> 2
10 tracks -> 2
25 -> 2
100 -> 2

'''

# Choose Background data

In [4]:
#get trackslist to eliminate that are not covered from the refset while evaluating
import itertools as it
import sys
import json
import re
import collections
import os
path='../../data'


filenames = os.listdir(path)[:750]

count=0

tracksSet=[]
pidLen=[]
pidDate=[]
trainingDataPids=[]

for filename in sorted(filenames):
    if filename.startswith("mpd.slice.") and filename.endswith(".json"):
        fullpath = os.sep.join((path, filename))
        f = open(fullpath)
        js = f.read()
        f.close()
        mpd_slice = json.loads(js)
        for playlist in mpd_slice['playlists']:
            playlistId=str(playlist['pid'])
            numTracks=playlist['num_tracks']
            lastMod=playlist['modified_at']
            trainingDataPids.append(playlistId)
            pidLen.append(numTracks)
            pidDate.append(lastMod)
            for track in playlist['tracks']:
                tracksSet.append(track['track_uri'])
            count=count+1
            if count % 10000 ==0:
                print 'processed' + str(count)
tracksSet=set(tracksSet)

processed10000
processed20000
processed30000
processed40000
processed50000
processed60000
processed70000
processed80000
processed90000
processed100000
processed110000
processed120000
processed130000
processed140000
processed150000
processed160000
processed170000
processed180000
processed190000
processed200000
processed210000
processed220000
processed230000
processed240000
processed250000
processed260000
processed270000
processed280000
processed290000
processed300000
processed310000
processed320000
processed330000
processed340000
processed350000
processed360000
processed370000
processed380000
processed390000
processed400000
processed410000
processed420000
processed430000
processed440000
processed450000
processed460000
processed470000
processed480000
processed490000
processed500000
processed510000
processed520000
processed530000
processed540000
processed550000
processed560000
processed570000
processed580000
processed590000
processed600000
processed610000
processed620000
processed630000
p

In [5]:
len(tracksSet), len(pidLen), len(pidDate)

(1963619, 750000, 750000)

# Save Background data info

In [176]:
import pickle
fileObj = open('filenamesBackGroundData.pkl', 'w')
pickle.dump(filenames, fileObj)
fileObj.close()

import pickle
fileObj = open('backgroundDataPids.pkl', 'w')
pickle.dump(trainingDataPids, fileObj)
fileObj.close()

In [6]:
import pickle
fileObj = open('backGroundtracksSet750.pkl', 'w')
pickle.dump(tracksSet, fileObj)
fileObj.close()

# Create Training Data Splits

In [8]:
trainingfilenames = os.listdir(path)[750:]

count=0

trainingCandidateMeta={}
for filename in sorted(trainingfilenames):
    if filename.startswith("mpd.slice.") and filename.endswith(".json"):
        fullpath = os.sep.join((path, filename))
        f = open(fullpath)
        js = f.read()
        f.close()
        mpd_slice = json.loads(js)
        for playlist in mpd_slice['playlists']:
            playlistId=playlist['pid']
            numTracks=playlist['num_tracks']
            lastMod=playlist['modified_at']
            tracksList=[]
            for track in playlist['tracks']:
                tracksList.append(track['track_uri'])
            overlapWithBackground=len(set(tracksList)&tracksSet)
            overlapWithBackground1=len(set(tracksList[:1])&tracksSet)
            overlapWithBackground5=len(set(tracksList[:5])&tracksSet)
            overlapWithBackground10=len(set(tracksList[:10])&tracksSet)
            pOverlap=overlapWithBackground*1.0/numTracks
            trainingCandidateMeta[playlistId]=[numTracks,lastMod,overlapWithBackground,overlapWithBackground1,overlapWithBackground5,overlapWithBackground10,pOverlap]
            count=count+1
            if count % 10000 ==0:
                print 'processed' + str(count)

processed10000
processed20000
processed30000
processed40000
processed50000
processed60000
processed70000
processed80000
processed90000
processed100000
processed110000
processed120000
processed130000
processed140000
processed150000
processed160000
processed170000
processed180000
processed190000
processed200000
processed210000
processed220000
processed230000
processed240000
processed250000


In [9]:
trainingCandidateMeta.items()[100] #(855477, [226, 1509494400, 224, 1, 5, 10, 0.9911504424778761])

(855477, [226, 1509494400, 224, 1, 5, 10, 0.9911504424778761])

In [10]:
import pickle
fileObj = open('trainingTestingCandidateMeta750.pkl', 'w')
pickle.dump(trainingCandidateMeta, fileObj)
fileObj.close()

# Inspect Histograms

In [169]:
selectHist=[]
Hist=[]
for k, v in trainingCandidateMeta.items():
    if v[-1]==1.0:
        selectHist.append(v[0])
    Hist.append(v[0])

In [170]:
len(selectHist),len(Hist)

(138471, 250000)

In [171]:
import numpy as np
hist_1, _ = np.histogram(selectHist, bins=50)
hist_2, _ = np.histogram(Hist, bins=50)

In [172]:
250000*1.0/138471

1.8054321843562913

In [173]:
minima = np.minimum(hist_1*1.8, hist_2)#normalise by the ratio
intersection = np.true_divide(np.sum(minima), np.sum(hist_2))
intersection#print histogram intersections

0.93217840000000007

In [177]:
len(list(filter(lambda x: x > 100, selectHist))) , len(list(filter(lambda x: x > 100, Hist)))

(12969, 53905)

In [178]:
len(list(filter(lambda x: x > 25 and x< 100, selectHist))) , len(list(filter(lambda x: x > 25 and x< 100, Hist)))

(76078, 133473)

# Generate Splits

In [179]:
training100CandidatePid=[]
training100RandCandidatePid=[]
training25CandidatePid=[]
training25RandCandidatePid=[]
training10CandidatePid=[]
training5CandidatePid=[]
training1CandidatePid=[]
for k, v in trainingCandidateMeta.items():
    if v[-1]==1.0:
        if v[0]>100:
            training100CandidatePid.append(k)
        if v[0]>25:
            training25CandidatePid.append(k)
        if v[0]>10:
            training10CandidatePid.append(k)
        if v[0]>5:
            training5CandidatePid.append(k)
        training1CandidatePid.append(k)
        
    if v[0]>100 and v[-1]>=0.99:
        training100RandCandidatePid.append(k)
    if v[0]>25 and v[-1]>=0.99:
        training25RandCandidatePid.append(k)

len(training100CandidatePid),len(training25CandidatePid), len(training10CandidatePid), len(training5CandidatePid), len(training1CandidatePid)

(12969, 89491, 128457, 137258, 138471)

In [180]:
len(training100RandCandidatePid),len(training25RandCandidatePid)

(23800, 100546)

In [181]:
#shuffle the candidate pids
from random import shuffle
shuffle(training100CandidatePid)
shuffle(training25CandidatePid)
shuffle(training10CandidatePid)
shuffle(training5CandidatePid)
shuffle(training1CandidatePid)
shuffle(training100RandCandidatePid)
shuffle(training25RandCandidatePid)
training100Pid=training100CandidatePid[:5000]
training25Pid= list(set(training25CandidatePid) - set(training100Pid))[:5000]
training10Pid= list(set(training10CandidatePid) - set(training100Pid+training25Pid))[:10000]
training5Pid= list(set(training5CandidatePid) - set(training100Pid+training25Pid+training10Pid))[:10000]
training1Pid= list(set(training1CandidatePid) - set(training100Pid+training25Pid+training10Pid+training5Pid))[:5000]
trainingOnlyTitlePid= list(set(training1CandidatePid) - set(training100Pid+training25Pid+training10Pid+training5Pid+training1Pid))[:5000]
training100RandPid= list(set(training100RandCandidatePid) - set(training100Pid+training25Pid+training10Pid+training5Pid+training1Pid+trainingOnlyTitlePid))[:5000]
training25RandPid= list(set(training25RandCandidatePid) - set(training100Pid+training25Pid+training10Pid+training5Pid+training1Pid+trainingOnlyTitlePid+training100RandPid))[:5000]


In [182]:
len(training100Pid),len(training100RandPid),len(training25Pid),len(training25RandPid),len(training10Pid),len(training5Pid),len(training1Pid),len(trainingOnlyTitlePid)

(5000, 5000, 5000, 5000, 10000, 10000, 5000, 5000)

In [186]:
trainingCandidateMeta[training10Pid[100]]

[112, 1508457600, 112, 1, 5, 10, 1.0]

In [184]:
AllTrain=training100Pid+training25Pid+training10Pid+training5Pid+training1Pid+trainingOnlyTitlePid+training100RandPid+training25RandPid

In [185]:
leftOvers=set(trainingCandidateMeta.keys())-set(AllTrain)

In [198]:
'''
leftOverPlaylistIdsAfterBackgroundAndTrain1.pkl
training100Pid.pkl
training100RandPid.pkl
training10Pid.pkl
training1Pid.pkl
training25Pid.pkl
training25RandPid.pkl
training5Pid.pkl
trainingOnlyTitlePid.pkl
'''

import pickle
fileObj = open('training100Pid.pkl', 'w')
pickle.dump(training100Pid, fileObj)
fileObj.close()

In [199]:
len(set(leftOvers))

200000

# Create Test Data

In [1]:
import pickle
leftOverPids= pickle.load(open('./SplitsInformation/leftOverPlaylistIdsAfterBackgroundAndTrain1.pkl','rb'))

In [ ]:
#Create splits for testing; 500 in each test track

In [16]:
testing100CandidatePid=[]
testing100RandCandidatePid=[]
testing25CandidatePid=[]
testing25RandCandidatePid=[]
testing10CandidatePid=[]
testing5CandidatePid=[]
testing1CandidatePid=[]
for k, v in trainingCandidateMeta.items():
    if k in leftOverPids:
        if v[-1]==1.0:
            if v[0]>100:
                testing100CandidatePid.append(k)
            if v[0]>25:
                testing25CandidatePid.append(k)
            if v[0]>10:
                testing10CandidatePid.append(k)
            if v[0]>5:
                testing5CandidatePid.append(k)
            testing1CandidatePid.append(k)

        if v[0]>100 and v[-1]>=0.99:
            testing100RandCandidatePid.append(k)
        if v[0]>25 and v[-1]>=0.99:
            testing25RandCandidatePid.append(k)

len(testing100CandidatePid),len(testing25CandidatePid), len(testing10CandidatePid), len(testing5CandidatePid), len(testing1CandidatePid)

(3878, 55371, 84874, 91616, 92561)

In [19]:
from random import shuffle
shuffle(testing100CandidatePid)
shuffle(testing25CandidatePid)
shuffle(testing10CandidatePid)
shuffle(testing5CandidatePid)
shuffle(testing1CandidatePid)
shuffle(testing100RandCandidatePid)
shuffle(testing25RandCandidatePid)
testing100Pid=testing100CandidatePid[:500]
testing25Pid= list(set(testing25CandidatePid) - set(testing100Pid))[:500]
testing10Pid= list(set(testing10CandidatePid) - set(testing100Pid+testing25Pid))[:1000]
testing5Pid= list(set(testing5CandidatePid) - set(testing100Pid+testing25Pid+testing10Pid))[:1000]
testing1Pid= list(set(testing1CandidatePid) - set(testing100Pid+testing25Pid+testing10Pid+testing5Pid))[:500]
testingOnlyTitlePid= list(set(testing1CandidatePid) - set(testing100Pid+testing25Pid+testing10Pid+testing5Pid+testing1Pid))[:500]
testing100RandPid= list(set(testing100RandCandidatePid) - set(testing100Pid+testing25Pid+testing10Pid+testing5Pid+testing1Pid+testingOnlyTitlePid))[:500]
testing25RandPid= list(set(testing25RandCandidatePid) - set(testing100Pid+testing25Pid+testing10Pid+testing5Pid+testing1Pid+testingOnlyTitlePid+testing100RandPid))[:500]


In [23]:
len(testing100Pid),len(testing100RandPid),len(testing25Pid),len(testing25RandPid),len(testing10Pid),len(testing5Pid),len(testing1Pid),len(testingOnlyTitlePid)



(500, 500, 500, 500, 1000, 1000, 500, 500)

In [26]:
trainingCandidateMeta[testing10Pid[398]]

[45, 1455926400, 45, 1, 5, 10, 1.0]

In [27]:
AllTesting=testing100Pid+testing25Pid+testing10Pid+testing5Pid+testing1Pid+testingOnlyTitlePid+testing100RandPid+testing25RandPid



In [28]:
leftOversForTelescoping=leftOverPids-set(AllTesting)

In [60]:
import pickle
fileObj = open('leftOversForTelescoping.pkl', 'w')
pickle.dump(leftOversForTelescoping, fileObj)
fileObj.close()

import pickle
fileObj = open('AlltestingPids.pkl', 'w')
pickle.dump(AllTesting, fileObj)
fileObj.close()


In [40]:
telescoping100CandidatePid=[]
telescoping100RandCandidatePid=[]
telescoping25CandidatePid=[]
telescoping25RandCandidatePid=[]
telescoping10CandidatePid=[]
telescoping5CandidatePid=[]
telescoping1CandidatePid=[]
for k, v in trainingCandidateMeta.items():
    if k in leftOversForTelescoping:
        if v[-1]==1.0:
            if v[0]>100:
                telescoping100CandidatePid.append(k)
            if v[0]>25:
                telescoping25CandidatePid.append(k)
            if v[0]>10:
                telescoping10CandidatePid.append(k)
            if v[0]>5:
                telescoping5CandidatePid.append(k)
            telescoping1CandidatePid.append(k)

        if v[0]>100 and v[-1]>=0.99:
            telescoping100RandCandidatePid.append(k)
        if v[0]>25 and v[-1]>=0.99:
            telescoping25RandCandidatePid.append(k)

len(telescoping100CandidatePid),len(telescoping25CandidatePid), len(telescoping10CandidatePid), len(telescoping5CandidatePid), len(telescoping1CandidatePid)

(3042, 52287, 80474, 87071, 87993)

In [42]:
from random import shuffle
shuffle(telescoping100CandidatePid)
shuffle(telescoping25CandidatePid)
shuffle(telescoping10CandidatePid)
shuffle(telescoping5CandidatePid)
shuffle(telescoping1CandidatePid)
shuffle(telescoping100RandCandidatePid)
shuffle(telescoping25RandCandidatePid)
telescoping100Pid=telescoping100CandidatePid[:3000]
telescoping25Pid= list(set(telescoping25CandidatePid) - set(telescoping100Pid))[:3000]
telescoping10Pid= list(set(telescoping10CandidatePid) - set(telescoping100Pid+telescoping25Pid))[:6000]
telescoping5Pid= list(set(telescoping5CandidatePid) - set(telescoping100Pid+telescoping25Pid+telescoping10Pid))[:6000]
telescoping1Pid= list(set(telescoping1CandidatePid) - set(telescoping100Pid+telescoping25Pid+telescoping10Pid+telescoping5Pid))[:3000]
telescopingOnlyTitlePid= list(set(telescoping1CandidatePid) - set(telescoping100Pid+telescoping25Pid+telescoping10Pid+telescoping5Pid+telescoping1Pid))[:3000]
telescoping100RandPid= list(set(telescoping100RandCandidatePid) - set(telescoping100Pid+telescoping25Pid+telescoping10Pid+telescoping5Pid+telescoping1Pid+telescopingOnlyTitlePid))[:3000]
telescoping25RandPid= list(set(telescoping25RandCandidatePid) - set(telescoping100Pid+telescoping25Pid+telescoping10Pid+telescoping5Pid+telescoping1Pid+telescopingOnlyTitlePid+telescoping100RandPid))[:3000]


In [43]:
len(telescoping100Pid),len(telescoping100RandPid),len(telescoping25Pid),len(telescoping25RandPid),len(telescoping10Pid),len(telescoping5Pid),len(telescoping1Pid),len(telescopingOnlyTitlePid)


(3000, 3000, 3000, 3000, 6000, 6000, 3000, 3000)

In [44]:
AllTelescoping=telescoping100Pid+telescoping25Pid+telescoping10Pid+telescoping5Pid+telescoping1Pid+telescopingOnlyTitlePid+telescoping100RandPid+telescoping25RandPid


In [59]:
import pickle
fileObj = open('telescoping25RandPid.pkl', 'w')
pickle.dump(telescoping25RandPid, fileObj)
fileObj.close()

In [48]:
leftOversAfterTelescoping=leftOversForTelescoping-set(AllTelescoping)

In [50]:
len(leftOversAfterTelescoping)

165000

In [51]:
import pickle
fileObj = open('leftOversAfterTelescoping.pkl', 'w')
pickle.dump(leftOversAfterTelescoping, fileObj)
fileObj.close()

import pickle
fileObj = open('AllTelescoping.pkl', 'w')
pickle.dump(AllTelescoping, fileObj)
fileObj.close()